# Tests for Tiktok Research API

## Config

In [1]:
from dotenv import load_dotenv
import os
import requests
import json
from typing import Optional, Dict, Any, List

load_dotenv()

True

In [2]:
TIKTOK_CLIENT_ID = os.getenv("TIKTOK_CLIENT_ID")
assert TIKTOK_CLIENT_ID is not None, "bad .env"
TIKTOK_CLIENT_SECRET = os.getenv("TIKTOK_CLIENT_SECRET")
assert TIKTOK_CLIENT_SECRET is not None, "bad .env"
TIKTOK_CLIENT_KEY = os.getenv("TIKTOK_CLIENT_KEY")
assert TIKTOK_CLIENT_KEY is not None, "bad .env"

In [3]:
tiktok_account = "coldplay"

## Get access token

In [4]:
def get_access_token(client_key: str, client_secret: str) -> Optional[Dict[str, Any]]:
    """
    Requests an access token using the Client Credentials grant flow for the
    TikTok Open API.

    Args:
        client_key: Your application's client key.
        client_secret: Your application's client secret.

    Returns:
        A dictionary containing the token information (e.g., access_token,
        token_type, expires_in, scope) on success, or None on failure.
    """
    url = 'https://open.tiktokapis.com/v2/oauth/token/'

    # Headers specific for this OAuth endpoint
    headers = {
        # The Content-Type header is usually handled by requests when using 'data',
        # but we set it explicitly as good practice based on the curl command.
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cache-Control': 'no-cache'
    }

    # Form-encoded data payload
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
        'scope': 'research.data.basic'
    }

    print("Attempting to request access token...")

    try:
        # Send the POST request. Using 'data' sends the content as
        # application/x-www-form-urlencoded.
        response = requests.post(
            url,
            headers=headers,
            data=data,
            timeout=10
        )

        # Check for HTTP errors
        response.raise_for_status()

        # Parse the JSON response
        response_data = response.json()

        # Check if the token was successfully generated (it should contain 'access_token')
        if 'access_token' in response_data:
            print("Access token successfully received.")
            return response_data
        else:
            print(f"API returned success status but no token: {response_data.get('error_description', response.text)}")
            return None


    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error during token request: {errh}")
        # The response text often contains the specific API error message
        if 'response' in locals() and response.text:
            print(f"API Response Content: {response.text}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected request error occurred: {err}")
    except json.JSONDecodeError:
        print("Failed to decode JSON response from API during token request.")

    return None

In [5]:
access_token_info = get_access_token(TIKTOK_CLIENT_KEY, TIKTOK_CLIENT_SECRET)
access_token = access_token_info["access_token"]


Attempting to request access token...
Access token successfully received.


a## General info

In [6]:
def get_tiktok_user_info(username: str, auth_token: str) -> Optional[Dict[str, Any]]:
    """
    Fetches public profile information for a given TikTok username using the
    TikTok Research API v2.

    Args:
        username: The TikTok username (e.g., 'joe123456').
        auth_token: Your valid Bearer Authorization token.

    Returns:
        A dictionary containing the user's information on success, or None on failure.
    """
    url = 'https://open.tiktokapis.com/v2/research/user/info/'

    # Define the fields we want to retrieve
    fields_to_query = (
        # ',display_name'
        # ',bio_description'
        # ',avatar_url'
        'is_verified'
        ',follower_count'
        ',following_count'
        ',likes_count'
        ',video_count'
    )

    # Headers for authorization and content type
    headers = {
        'Authorization': f'Bearer {auth_token}',
        'Content-Type': 'application/json'
    }

    # Query parameters (fields list)
    params = {
        'fields': fields_to_query
    }

    # JSON request body (username)
    data = {
        'username': username
    }

    print(f"Attempting to fetch data for username: {username}...")

    response = None

    try:
        # Send the POST request
        response = requests.post(
            url,
            headers=headers,
            params=params,
            json=data
        )

        # Check for HTTP errors (4xx or 5xx status codes)
        response.raise_for_status()

        # Parse the JSON response
        response_data = response.json()["data"]

        # Check if the user is verified
        if not response_data["is_verified"]:
            print("User is not verified.")
            return None
        return response_data

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
        if response is not None:
            print(f"API Response Content: {response.text}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected request error occurred: {err}")
    except json.JSONDecodeError:
        print("Failed to decode JSON response from API.")

    return None

In [7]:
get_tiktok_user_info(tiktok_account, access_token)

Attempting to fetch data for username: coldplay...


{'following_count': 37,
 'is_verified': True,
 'likes_count': 87842758,
 'video_count': 391,
 'follower_count': 8121541}

## Get following list

In [8]:
def get_all_following_list(username: str, auth_token: str, max_count: int = 100) -> Optional[List[Dict[str, Any]]]:
    """
    Fetches the complete list of users that a given TikTok username is following,
    handling pagination automatically.

    Args:
        username: The TikTok username (e.g., 'joe123456').
        auth_token: Your valid Bearer Authorization token.
        max_count: The maximum number of users to fetch per API request (max 100).

    Returns:
        A list of dictionaries, where each dictionary represents a user being followed.
    """
    url = 'https://open.tiktokapis.com/v2/research/user/following/'
    all_following = []
    cursor = None

    # Define the fields we want to retrieve (using the same fields as the info query)
    fields_to_query = (
        'display_name,bio_description,avatar_url,is_verified,'
        'follower_count,following_count,likes_count,video_count'
    )

    headers = {
        'Authorization': f'Bearer {auth_token}',
        'Content-Type': 'application/json'
    }

    # Query parameters (fields list)
    params = {
        'fields': fields_to_query
    }

    print(f"Starting pagination to fetch all users followed by: {username}...")

    response = None

    while True:
        data = {
            'username': username,
            'max_count': max_count
        }

        if cursor:
            data['cursor'] = cursor

        try:
            # Send the POST request
            response = requests.post(
                url,
                headers=headers,
                params=params,
                json=data,
                timeout=10
            )

            response.raise_for_status()
            response_data = response.json()
            data_block = response_data.get('data', {})

            # Check for API-level errors
            if response_data.get('error', {}) and response_data['error'].get('code') != "ok":
                print(f"API Error in response: {response_data.get('error')}")
                # Print specific error detail if available
                if response_data.get('error', {}).get('message'):
                    print(f"Error Message: {response_data['error']['message']}")
                return None

            # FIX: Changed key from 'users' to 'user_following' based on provided JSON structure
            users = data_block.get('user_following', [])
            all_following.extend(map(lambda user: user['username'], users))

            has_more = data_block.get('has_more', False)
            cursor = data_block.get('cursor')

            print(f"Fetched {len(users)} users in this page. Total collected: {len(all_following)}. Has more: {has_more}")

            if not has_more or not cursor or cursor == -1: # Added check for cursor being -1
                break

        except requests.exceptions.HTTPError as errh:
            print(f"HTTP Error during following list fetch: {errh}")
            if response is not None:
                print(f"API Response Content: {response.text}")
            return None
        except requests.exceptions.RequestException as err:
            print(f"An unexpected request error occurred: {err}")
            return None
        except json.JSONDecodeError:
            print("Failed to decode JSON response from API.")
            return None

    print(f"Finished fetching. Total following users found: {len(all_following)}")
    return all_following

In [9]:
get_all_following_list(tiktok_account, access_token)

Starting pagination to fetch all users followed by: coldplay...
Fetched 36 users in this page. Total collected: 36. Has more: False
Finished fetching. Total following users found: 36


['igaswiatek_official',
 'beargrylls',
 'akashasky',
 'twice_tiktok_official',
 'willstevoe',
 'tinistoessel',
 'paralympics',
 'pokemonnewstv',
 'divinelyrcs',
 'c4news',
 'qvc',
 'legitpat',
 'sum41',
 'pattylat0',
 'ayrastarr',
 'roses_are_rosie',
 'mr.mr.muscle',
 'itsyujen',
 'annaleemedia',
 'selenagomez',
 'lizzo',
 'shakira',
 'mariahdez19',
 'theethanfields',
 'imkevinhart',
 'red',
 'glastonbury',
 '__mariayakovleva__',
 'alessiasmusic',
 'johnlegend',
 'progresspop',
 'therock',
 'callherdaddy',
 'oxfam',
 'tovelo',
 'dualipaofficial']